# Trabalho Prático I
Alunas: Cecília Kind e Selene Melo

### Resumo:
O programa a seguir realiza a compressão de uma imagem a partir Codificação Preditiva sem perdas seguida da codificação de Huffman. O trabalho é divido em algumas etapas que serão abordadas passo a passo:

- 1: Obtenção e especificaçao da imagem orginal
- 2: Aplicação da codificação preditiva sem perda
- 3: Aplicação da codificação de Huffman
- 4: Armazenamaneto da imagem
- 5: Recuperação da imagem armazenada
- 6: Decodificação do código de Huffman
- 7: Recuperação da imagem a partir da matriz de erro
- 8: Análise final

### Bibliotecas utilizadas:
Obs: a estrutura BitArray foi importada da biblioteca bitstring, que pode ser instalada com o comando comentado abaixo.

In [ ]:
import os, struct
from typing import NamedTuple, List
from collections import Counter
from collections import defaultdict
from pprint import pprint

import numpy as np
import cv2
import json
from matplotlib import pyplot as plt

#!pip install --user bitstring
from bitstring import BitArray

%matplotlib inline

### Funções e estruturas utilizadas:

- Entropia, RMSE e impressão da imagem

In [ ]:
def imprime_imagem(imagem, titulo):
    plt.title(titulo)
    plt.imshow(imagem, cmap = 'gray')
    plt.show()

def calcula_entropia(imagem):
    freq = dict(sum(map(Counter, imagem), Counter()))
    prob = [freq[k]/(height* width) for k in freq]   
    entropia = 0
    for k in prob:
        if k != 0:
            entropia = entropia + k*np.log2(k)
    return -entropia

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

- Codificação e decodificação preditiva

In [ ]:
def predicao_erro(imagem, height, width):
    size = height, width
    img_pred_erro = [[0 for c in range(size[1])] for r in range(size[0])]

    for row in range(0,height):
        img_pred_erro[row][0] = int(img_original[row][0])

    for row in range(0,height):
      for col in range(1, width):
        img_pred_erro[row][col] = int(img_original[row][col]) - int(img_original[row][col-1])
    
    return img_pred_erro

def recupera_erro(img_decomp, height, width ):
    for row in range(0,height):
        for col in range(1, width):
            img_decomp[row][col] = int(img_decomp[row][col]) + int(img_decomp[row][col-1])
    return img_decomp

- Codificação e decodificação de Huffman

In [ ]:
class SymCode(NamedTuple):
    old: int
    new: str

class ProbSymCodes(NamedTuple):
    prob: float
    codes: List[SymCode]

def codif_Huffman(imagem, height, width):
    # calculando as frequencias
    frequencia = dict(sum(map(Counter, imagem), Counter()))
    lista_codigos = [ProbSymCodes(prob = frequencia[k]/(height* width), codes = [SymCode(k, "")]) for k in frequencia]
    lista_codigos = sorted(lista_codigos)
    
    while(len (lista_codigos) > 1):

        if type(lista_codigos[-1]) == ProbSymCodes: lo = lista_codigos.pop(0)
        if type(lista_codigos[-1]) == ProbSymCodes: hi = lista_codigos.pop(0)

        for idx, code in enumerate(lo.codes):
            lo.codes[idx] = lo.codes[idx]._replace(new='0'+ lo.codes[idx].new)

        for idx, code in enumerate(hi.codes):
            hi.codes[idx] = hi.codes[idx]._replace(new='1'+ hi.codes[idx].new)

        lista_codigos.insert(0, ProbSymCodes(prob = lo.prob + hi.prob, codes = lo.codes + hi.codes))
        lista_codigos = sorted(lista_codigos)

    return dict((code.old, code.new) for code in lista_codigos[-1][-1])

def decod_Huffman(img_bitstr, cod_dic, height, width):
    
    i = 0
    j = 1

    size = height, width
    img_decomp = np.zeros(size)

    for row in range(0,height):
        for col in range(0,width):
            while img_bitstr[i:j] not in cod_dic:
                j += 1        
            img_decomp[row][col] = cod_dic[img_bitstr[i:j]]
            i = j
            j += 1
            
    return img_decomp

def eficiencia_Huffman(resultado_cod, entropia, height, width):
    return entropia/(len(resultado_cod)/(height*width))

- Armazenamaneto e recuperação da nova imagem

In [ ]:
def armazena_imagem(imagem, dicionario_codigos, matriz_codigos):
    # WxH
    img_size = struct.pack('!2I', imagem.shape[1], imagem.shape[0])
    with open('compressed.txt', 'wb') as comp:
        comp.write(img_size)

        # dicionario
        dict_bin = json.dumps(dicionario_codigos).encode('utf-8')
        dict_len = len(dict_bin)
        dicionario_size = struct.pack('!I', dict_len)
        comp.write(dicionario_size)
        comp.write(dict_bin)

        #imagem
        padding = 8 - len(matriz_codigos)%8
        padding_bin = struct.pack('!I', padding)
        comp.write(padding_bin)
        matriz_codigos = matriz_codigos + ('0' * padding)
        assert(len(matriz_codigos)%8 == 0)

        enc = BitArray(bin='')
        cnt = 0
        while(cnt < len(matriz_codigos)):
            enc = enc + BitArray(bin=matriz_codigos[cnt:cnt+8])
            cnt += 8

        comp.write(enc.bytes)
        return 'compressed.txt'
    
def recupera_dados(nome_arquivo):
    with open(nome_arquivo, 'rb') as comp:
        oito_bytes = comp.read(8)

        w_comp = struct.unpack('!I', oito_bytes[:4])[0]
        h_comp = struct.unpack('!I', oito_bytes[4:])[0]

        quatro_bytes = comp.read(4)
        tam_dict = struct.unpack('!I', quatro_bytes)[0]

        quatro_bytes = comp.read(tam_dict)
        cod_dict = {int(k): v for k, v in json.loads(quatro_bytes).items()}

        quatro_bytes = comp.read(4)
        recpadding = struct.unpack('!I', quatro_bytes)[0]

        img_binary = comp.read()
        img_bitstr = BitArray(img_binary).bin[:-recpadding]
    return h_comp, w_comp, cod_dict, img_bitstr

### 1: Obtenção e especificação da imagem orginal
Recebe o nome do arquivo da imagem e calcula as informações iniciais

In [ ]:
infile = input("Entre o nome do arquivo de imagem: ")

In [ ]:
img_original = cv2.imread(infile,0)

height, width =  img_original.shape
size = height, width
imprime_imagem(img_original, "Imagem Original")

entropia_original = calcula_entropia(img_original)
tamanho_imagem_bytes = entropia_original*height* width/8
tam_arquivo_original = os.path.getsize(infile)
print("Entropia imagem original: ", entropia_original)
print("Tamanho da imagem em bytes: ", tamanho_imagem_bytes)
print("Tamanho do arquivo da imagem em bytes: ", tam_arquivo_original, "bytes \n")

### 2: Aplicação da codificação preditiva sem perda

Este tipo de codificação explora a redundância interpixel, que explora a característica de que pixels vizinhos normalmente possuem alguma realção. O processo consiste na suposição do valor de cada pixel a partir do pixel vizinho e no registro da diferença entre o valor previsto e o valor real em uma matriz de erro.  No caso, a função f(x+1) = f(x) foi utilizada como função preditora. A vantagem de se aplicar este método é diminuir a entropia da imagem se a redundância interpixel da imagem for alta. Assumindo que pixels próximos são parecidos, os valores na matriz de erro serão baixos, gerando um número de tons de cinza distintos menor, pois os valores altos (como a diferença entre bordas) aparecem com menos frequência.

A construção da matriz de erro e o cálculo da entropia são realizados abaixo. Se o valor da entropia for inferior à entropia da imagem original a codificação de Huffman será efetiva. Caso contrário, a imagem escolhida não é uma boa opção para este método. A taxa de compressão máxima considerando apenas a estrutura das imagens é calculada abaixo. Se a taxa for acima de 1 a entropia diminuiu, caso contrário a predição de erro não é eficiente para a imagem escolhida.

In [ ]:
img_pred_erro = predicao_erro(img_original, height, width)
entropia_erro = calcula_entropia(img_pred_erro)
imprime_imagem(img_pred_erro, "Representação matriz de erro")
print("Entropia matriz de erro: ", entropia_erro, "\n")
print("Taxa de compressão máxima: ", entropia_original/entropia_erro)

### 3: Aplicação da codificação de Huffman

A codificação de Huffman utiliza a redundância de codificação para reduzir o tamanho dos códigos utilizados para representar níveis de cinza contidos na imagem. A imagem original possui tons de cinza que variam entre 0 e 255, sendo representados cada um por 8bits. A ideia é reduzir o código de um tom de cinza que tem probabilidade alta e utilizar uma sequência maior para os que aparecem pouco. 

O algoritmo consiste na criação de uma árvore em que os símbolos com menor probabilidade são incluídos antes e a árvore é construída de baixo para cima. No fim, o caminho na árvore da raiz ao símbolo te dá a codificação. Na implementação, uma lista foi utilizada como estrutura no lugar da árvore para realizar a ordenação das probabilidades, mas a ideian central do algoritmo original foi mantida.

Este método só é eficiente quando a entropia é menor do que o número médio original de bits por pixels. Neste caso, a codificação preditiva foi aplicada anteriormente para diminuir a entropia e aumentar a eficiência desta codificação.

In [ ]:
dicionario_codigos = codif_Huffman(img_pred_erro,height, width)
#print(dicionario_codigos)

matriz_codigos = ""
for row in range(0, height):
    for col in range(0, width):
        matriz_codigos = matriz_codigos + dicionario_codigos[img_pred_erro[row][col]]

print("Eficiência da codificação de Huffman: ", eficiencia_Huffman(matriz_codigos, entropia_erro, height, width), "\n")
        
print("Subsituição dos códigos originais pela nova codificação: \n\n",matriz_codigos)

### 4: Armazenamaneto da imagem
Os dados foram convertidos para bytes com auxílio da estrutura bitArray e do módulo Struct (que interpreta bytes como dados binários compactados), e foram armazenados em um arquivo de texto da seguinte forma:
- largura (4 bytes)
- altura (4 bytes)
- dict_len: Número de bytes ocupados pelo dicionário de Huffman (4 bytes)
- Dicionário de Huffman (dict_len bytes)
- "padding" (4 bytes) (necessário para tornar a sequência da imagem um múltiplo de 8)
- imagem com os novos códigos (restante do arquivo)

In [ ]:
nome_arquivo = armazena_imagem(img_original, dicionario_codigos, matriz_codigos)
print(nome_arquivo)

### 5: Recuperação da imagem armazenada
Seguindo a ordem de armazenamento citada acima, a função recupera cada um dos dados em bytes e converte para o formato original.

In [ ]:
r_height, r_width, cod_dict, img_cod = recupera_dados(nome_arquivo)
print(r_height,r_width,"\n\n", cod_dict, "\n\n", img_cod)

### 6: Decodificação do código de Huffman
Uma vez que temos o dicionário dos códigos de Huffman e a sequência correspondente à matriz codificada, podemos agora realizar o inverso de Huffman, identificando cada código na sequência e substituindo cada posição na matriz de recuperação pelo símbolo original. A matriz resultante é a matriz de erro sem nehuma perda.

In [ ]:
cod_dict_inv = {v: k for k, v in cod_dict.items()}
imagem_recuperada = decod_Huffman(img_cod, cod_dict_inv, r_height, r_width)
imprime_imagem(imagem_recuperada, "Imagem de erro recuperada")

### 7: Recuperação da imagem a partir da matriz de erro
A partir da matriz de erro podemos recuperar a imagem aplicando o inverso da função preditiva aplicada no início e computando o erro. Neste passo também não temos perda de informação pois a primeira coluna manteve os valores originais da imagem e o erro não foi quantizado. O resultado desejado é uma matriz semelhante a imagem original.

In [ ]:
imagem_recuperada = recupera_erro(imagem_recuperada, r_height, r_width)
imprime_imagem(imagem_recuperada, "Imagem final")
entropia_imagem_recuperada = calcula_entropia(imagem_recuperada)
print("Entropia imagem recuperada: ", entropia_imagem_recuperada)

### 8: Análise final
A qualidade da compressão pode ser avaliada pelo erro médio quadrático (média da soma do quadrado das diferenças de cada ponto). No caso, como temos uma compressão sem perdas, o resultado do erro é sempre igual a zero.

Quanto a taxa de compressão final obtida, observamos que em algumas imagens testadas o tamanho do arquivo da imagem era muito superior ao tamanho calculado a partir da entropia e das dimensões da imagem (e que a diferença variava muito de imagem para imagem). 

Assim, apresentamos duas taxas de compressão finais: a primeira é a taxa do tamanho dos arquivos original e final, que foi elevada para todos os testes realizados. A segunda é taxa entre o tamanho em bytes da imagem (calculada a partir da entropia) e do arquivo final obtido. Esta taxa se mostrou em todos os casos testados semelhante à taxa de compressão prevista da entropia da imagem original e da entropia obtida com os métodos aplicados. Assim, a segunda taxa é melhor para avaliar a eficiência dos métodos aplicados.

In [ ]:
imprime_imagem(img_original, "Imagem original")
tam_arquivo_original = os.path.getsize(infile)
print("Tamanho do arquivo da imagem original: ", tam_arquivo_original, "bytes \n")
print("Tamanho da imagem original em bytes: ", tamanho_imagem_bytes)

imprime_imagem(imagem_recuperada, "Imagem final")
tam_compr = os.path.getsize(nome_arquivo)
print("Tamanho do arquivo armazenado: ", tam_compr, "bytes \n\n\n")

print("Taxa de compressão dos arquivos: ", tam_arquivo_original/tam_compr, "\n")
print("Taxa de compressão imagem original e arquivo final: ", tamanho_imagem_bytes/tam_compr, "\n")
print("Taxa de compressão máxima da entropia com os métodos aplicados: ", entropia_original/entropia_erro, "\n")
print("Eficiência da codificação de Huffman: ", eficiencia_Huffman(matriz_codigos, entropia_erro, height, width), "\n")
print("Erro médio quadrático: ", rmse(img_original, imagem_recuperada), '\n')